In [11]:
from IPython.display import clear_output

import datetime
import time
import os

from tools import *

from IPython.display import display
from IPython.core.display import HTML


play_beep()
display_full_output()

In [13]:
from ib_async import *

util.startLoop()

ib = IB()
ib.connect("127.0.0.1", 4001, clientId=13)
# util.logToConsole()

<IB connected to 127.0.0.1:4001 clientId=13>

In [15]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5), columns="bidSize bidPrice askPrice askSize".split())

# NQM2024 contract
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

def onTickerUpdate(ticker):
    bids = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)


# ticker.updateEvent += onTickerUpdate

# IB.sleep(15)

In [4]:
ib.cancelMktDepth(contract)

In [16]:
ib.positions(account="U10394496")

[Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685),
 Position(account='U10394496', contract=Option(conId=682674010, symbol='AAPL', lastTradeDateOrContractMonth='20241018', strike=150.0, right='P', multiplier='100', currency='USD', localSymbol='AAPL  241018P00150000', tradingClass='AAPL'), position=2.0, avgCost=244.69685),
 Position(account='U10394496', contract=Option(conId=669333316, symbol='NVDA', lastTradeDateOrContractMonth='20240719', strike=555.0, right='P', multiplier='100', currency='USD', localSymbol='NVDA  240719P00555000', tradingClass='NVDA'), position=1.0, avgCost=493.0494),
 Position(account='U10394496', contract=Option(conId=613471094, symbol='GOOGL', lastTradeDateOrContractMonth='20240920', strike=120.0, right='P', multiplier='100', currency='USD', localSym

In [17]:
# default reset values
open_trade = None
close_trade = None
# get a specific order by id
# trades = get_all_openorders(ib)
# sell2close = [trade for trade in trades if trade.order.permId == 1939815533][0]
# print_order(sell2close)

future = [
    pos for pos in ib.positions(account="U10394496") if pos.contract.symbol == "NQ"
][0]

tick_tp = 10
ticrement = 0.25

print(future)
print_account_summary(ib=ib)


Position(account='U10394496', contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', currency='USD', localSymbol='NQM4', tradingClass='NQ'), position=-3.0, avgCost=353809.73422575)
AccountType : INDIVIDUAL
Cushion : 0.762423
LookAheadNextChange : 1714507200
AccruedCash : 0.00
AvailableFunds : 49644.45
BuyingPower : 651032.43
EquityWithLoanValue : 162758.11
ExcessLiquidity : 162758.11
FullAvailableFunds : 49644.45
FullExcessLiquidity : 142172.39
FullInitMarginReq : 75480.97
FullMaintMarginReq : 68619.06
GrossPositionValue : 2683.26
InitMarginReq : 52836.68
LookAheadAvailableFunds : 49644.45
LookAheadExcessLiquidity : 142172.39
LookAheadInitMarginReq : 75480.97
LookAheadMaintMarginReq : 68619.06
MaintMarginReq : 48033.34
NetLiquidation : 213474.71
PreviousDayEquityWithLoanValue : 0.00
RegTEquity : 0.01
RegTMargin : 0.00
SMA : 623470.03
TotalCashValue : 210791.45


In [25]:
open_trade = None
close_trade = None

In [34]:
## SELL TO OPEN SCALP 0.1
########################################

sell_15_10_scalp = {
    "strategy": "SELL TO OPEN SCALP 15/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "SELL",
    "open_ref": "ask",
    "open_ticks": 15,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action":"BUY",
    "close_ref": "open_price_fill",
    "close_ticks": -10
    }

buy_10_10_scalp = {
    "strategy": "BUY TO OPEN SCALP 10/10",
    "contract": "NQM2024",
    "tick_increment": 0.25,
    "open_qty": 1,
    "open_type": "LIMIT",
    "open_action": "BUY",
    "open_ref": "bid",
    "open_ticks": -10,
    "close_qty": 1,
    "close_type": "LIMIT",
    "close_action":"SELL",
    "close_ref": "open_price_fill",
    "close_ticks": 10
    }

strategy_details = sell_15_10_scalp

print(strategy_details)

while True:
    clear_output(wait=True)  # Clear the output before printing new output

    # first order of the strategy
    if open_trade is None and close_trade is None:
        action = strategy_details['open_action']
        qty = strategy_details['open_qty']
        
        if strategy_details['open_ref'] == 'bid':
            price_ref = ticker.domBids[0].price
        elif strategy_details['open_ref'] == 'ask':
            price_ref = ticker.domAsks[0].price
        elif strategy_details['open_ref'] == 'mid':
            price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price)/2
        elif strategy_details['open_ref'] == 'last':
            raise Exception('Not implemented')

        lmtPrice = price_ref + strategy_details['open_ticks'] * strategy_details['tick_increment']
        print(f"Placing open trade: {action}, {strategy_details['open_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n")
        print()
        
        if strategy_details['open_type'] == "LIMIT":
            open_order = LimitOrder(
                action=action,
                totalQuantity=qty,
                lmtPrice=lmtPrice,
                account="U10394496",
            )
        else:
            raise Exception('Not implemented')
            
        open_trade = ib.placeOrder(contract, open_order)
        open_order_ts = datetime.datetime.now()
        play_beep(500, 500)

    print("OPEN ORDER::")
    print_order(open_trade)
    print()

    if open_trade is not None:
        if open_trade.orderStatus.status == "Submitted" and close_trade is None:
            print(
                f"Waiting to get filled on order #{open_trade.order.permId} ({open_trade.orderStatus.status})\n"
            )

            if datetime.datetime.now() - open_order_ts > datetime.timedelta(minutes=1):
                print("Cancelling order due to timeout:")
                ib.cancelOrder(open_trade.order)
                ib.sleep(1)
                print(open_trade.log)
                print()
            
        if open_trade.orderStatus.status == "Filled" and close_trade is None:
            action = strategy_details['close_action']
            qty = strategy_details['close_qty']
            
            if strategy_details['close_ref'] == 'open_price_fill':
                price_ref = open_trade.orderStatus.avgFillPrice
            if strategy_details['close_ref'] == 'bid':
                price_ref = ticker.domBids[0].price
            elif strategy_details['close_ref'] == 'ask':
                price_ref = ticker.domAsks[0].price
            elif strategy_details['close_ref'] == 'mid':
                price_ref = (ticker.domAsks[0].price + ticker.domBids[0].price)/2
            elif strategy_details['close_ref'] == 'last':
                raise Exception('Not implemented')
            
            lmtPrice = price_ref + strategy_details['close_ticks'] * strategy_details['tick_increment']
            print(f"Placing close trade: {action}, {strategy_details['close_type']}, totalQuantity {qty}, lmtPrice {lmtPrice}\n")
            
            if strategy_details['close_type'] == "LIMIT":
                close_order = LimitOrder(
                    action=action,
                    totalQuantity=qty,
                    lmtPrice=lmtPrice,
                    account="U10394496",
                )
            else:
                raise Exception('Not implemented')
            
            close_trade = ib.placeOrder(contract, close_order)
            ib.sleep(1)
        elif (
            open_trade.orderStatus.status == "Inactive"
            or open_trade.orderStatus.status == "Cancelled"
        ) and close_trade is None:
            print("***** order is inactive *****")
            print(open_trade.log)
            print("*****************************")
            open_trade = None

    print(f"CLOSE ORDER::")
    print_order(close_trade)
    print()

    if close_trade is not None:
        if close_trade.orderStatus.status == "Filled":
            play_beep()
            print("Close trade filled @ {}\n".format(close_trade.orderStatus.avgFillPrice))
            open_trade = None
            close_trade = None
            
            # small break before next trade
            for i in range(5):
                print(".", end="")
                time.sleep(1)
    
    print("ALL OPEN ORDERS::")
    print_all_openorders(ib=ib)
    print()

    # get current position of contract
    future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"][0]
    print(f'NQM2024: {ib.ticker(contract=contract).domBids[0].price}')    
    print(f"Position: {future.contract.symbol} {future.position} @ {future.avgCost/float(contract.multiplier)}"
    )
    acct_fields = ib.accountSummary(account="U10394496")
    print(f"NetLiquidation: {[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}")
    print(f"RealizedPnL: {[v for v in ib.accountValues() if v.tag == 'RealizedPnL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}")
    print(f"UnrealizedPnL: {[v for v in ib.accountValues() if v.tag == 'UnrealizedPnL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}")
    print(f"OptionMarketValue: {[v for v in ib.accountValues() if v.tag == 'OptionMarketValue' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}")
    print(f"TotalCashBalance: {[v for v in ib.accountValues() if v.tag == 'TotalCashBalance' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}")
    print(f"FuturesPNL: {[v for v in ib.accountValues() if v.tag == 'FuturesPNL' and v.currency == 'BASE' and v.account == 'U10394496'][0].value}")


    for i in range(3):
        print(".", end="")
        time.sleep(1)

Placing open trade: SELL, LIMIT, totalQuantity 1, lmtPrice 17530.0


OPEN ORDER::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	0	PendingSubmit		SELL	0.0	0.0		17530.0	

CLOSE ORDER::
None

ALL OPEN ORDERS::
symbol	permId		status		action	filled	remaining	lmtPrice
NQ	253829147	Submitted	SELL	0.0	1.0		17530.0	
NQ	253829147	Submitted	SELL	0.0	1.0		17530.0	
NQ	253828840	Submitted	SELL	0.0	1.0		17742.0	
NQ	1056387955	Submitted	SELL	0.0	2.0		21090.0	

NQM2024: 17527.25
Position: NQ 2.0 @ 17537.775056934373
NetLiquidation: 219268.9657
RealizedPnL: 29145.00
UnrealizedPnL: 575.87
OptionMarketValue: 3682.56
TotalCashBalance: 215586.4031
FuturesPNL: 30595.00
.

In [32]:
ib.accountValues()

[AccountValue(account='U10394496', tag='AccountCode', value='U10394496', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='AccountReady', value='true', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='ColumnPrio-C', value='2', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='ColumnPrio-P', value='5', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='ColumnPrio-S', value='1', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='Cushion', value='0.880709', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='DayTradesRemaining', value='-1', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='DayTradesRemainingT+1', value='-1', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='DayTradesRemainingT+2', value='-1', currency='', modelCode=''),
 AccountValue

In [57]:
future = [pos for pos in ib.positions() if pos.contract.symbol == "NQ"][0]
future.position

-4.0

In [60]:
ib.ticker(contract=contract).domBids[0].price

17700.5

In [ ]:
print(close_trade)
close_trade = ib.placeOrder(contract, close_order)
close_trade.log

None


[TradeLogEntry(time=datetime.datetime(2024, 4, 30, 16, 2, 16, 667472, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)]

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 317, reqId 6: Market depth data has been RESET. Please empty deep book contents before applying any new entries., contract: Contract(secType='FUT', conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ')
Peer closed connection.
Peer closed connection.


In [19]:
print_all_openorders(ib=ib)

symbol	permId		status		action	filled	remaining	lmtPrice
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	253828579	Submitted	BUY	0.0	1.0		17690.75	
NQ	1056387955	Submitted	SELL	0.0	2.0		21090.0	


In [31]:
trades = ib.reqAllOpenOrders()

trades.sort(key=lambda trade: trade.order.lmtPrice)

for trade in trades:
    orderstatus = trade.orderStatus
    order = trade.order

    if trade.contract.symbol != 'NQ':
        continue

    print(
        f"{trade.contract.symbol}\t{order.permId}\t{orderstatus.status}\t{order.action}\t{orderstatus.filled}\t{orderstatus.remaining}\t\t{order.lmtPrice}\t"
    )


NQ	1786380182	Submitted	BUY	0.0	1.0		17150.0	
NQ	1786380178	Submitted	BUY	0.0	1.0		17323.0	
NQ	1528433802	Submitted	BUY	0.0	3.0		17526.0	
NQ	1436152242	Submitted	BUY	0.0	1.0		17547.25	
NQ	1056387955	Submitted	SELL	0.0	2.0		20480.0	


In [47]:
print_account_summary(ib=ib)

AccountType : INDIVIDUAL
Cushion : 0.68701
LookAheadNextChange : 1714507200
AccruedCash : 0.00
AvailableFunds : 49644.45
BuyingPower : 584618.24
EquityWithLoanValue : 146154.56
ExcessLiquidity : 146154.56
FullAvailableFunds : 49644.45
FullExcessLiquidity : 118710.19
FullInitMarginReq : 100629.35
FullMaintMarginReq : 91481.23
GrossPositionValue : 2548.72
InitMarginReq : 70440.54
LookAheadAvailableFunds : 49644.45
LookAheadExcessLiquidity : 118710.19
LookAheadInitMarginReq : 100629.35
LookAheadMaintMarginReq : 91481.23
MaintMarginReq : 64036.86
NetLiquidation : 212740.14
PreviousDayEquityWithLoanValue : 0.00
RegTEquity : 0.01
RegTMargin : 0.00
SMA : 623470.03
TotalCashValue : 210191.42
